# Deeplabv3+ keras analysis

## Class imbalance

In [3]:
import os
import argparse
import time
import platform
import json
import warnings
import shutil
import random

import numpy as np
import cv2 as cv
from skimage.io import imread, imsave
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, Dropout
from tensorflow.keras.layers import (Concatenate
    , Lambda
    , Activation
    , AveragePooling2D
    , SeparableConv2D)
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras import optimizers
from tensorflow.keras.applications import MobileNetV2, Xception
from tensorflow.keras.utils import Sequence, GeneratorEnqueuer, OrderedEnqueuer
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import MeanIoU
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import math_ops, array_ops, confusion_matrix
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.python.keras.utils.data_utils import iter_sequence_infinite

#os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

# Constants.
DEBUG = True

MODE_TRAIN = 0
MODE_VAL = 1
MODE_TEST = 2

BASE_MODEL_MOBILENETV2 = 0
BASE_MODEL_XCEPTION = 1

RESOURCE_TYPE_PASCAL_VOC_2012 = 'pascal_voc_2012'
RESOURCE_TYPE_PASCAL_VOC_2012_EXT = 'pascal_voc_2012_ext'
RESOURCE_TYPE_GOOGLE_OPEN_IMAGES_V5 = 'google_open_images_v5'

GOIV5_SPECIFIC_SET = set(['Person', 'Cat', 'Dog', 'Car', 'Bus', 'Motorcycle', 'Bicyle'])


class MeanIoUExt(MeanIoU):
    """Calculate the mean IoU for one hot truth and prediction vectors."""

    def __init__(self, num_classes, accum_enable=True, name=None, dtype=None):
        super(MeanIoUExt, self).__init__(num_classes, name=name, dtype=dtype)
        self.accum_enable = accum_enable

    def update_state(self, y_true, y_pred, sample_weight=None):
        """Accumulated the confusion matrix statistics with one hot truth and prediction data.

        Parameters
        ----------
        y_true: Tensor or numpy array.
            One hot ground truth vectors.
        y_pred: Tensor or numpy array.
            One hot predicted vectors.
        sample_weight: Tensor.
            Optional weighting of each example. Defaults to 1. Can be a
            `Tensor` whose rank is either 0, or the same rank as `y_true`, and must
            be broadcastable to `y_true`.

        Returns
        -------
        Update operator.
            Operator
        """
        # Convert one hot vectors and labels.
        y_pred = K.argmax(y_pred)

        y_true = math_ops.cast(y_true, self._dtype)
        y_pred = math_ops.cast(y_pred, self._dtype)

        # Flatten the input if its rank > 1.
        if y_pred.shape.ndims > 1:
            y_pred = array_ops.reshape(y_pred, [-1])

        if y_true.shape.ndims > 1:
            y_true = array_ops.reshape(y_true, [-1])

        if sample_weight is not None and sample_weight.shape.ndims > 1:
            sample_weight = array_ops.reshape(sample_weight, [-1])

        # Accumulate the prediction to current confusion matrix.
        current_cm = confusion_matrix.confusion_matrix(
            y_true,
            y_pred,
            self.num_classes,
            weights=sample_weight,
            dtype=dtypes.float64)
        return self.total_cm.assign_add(current_cm) if self.accum_enable \
            else self.total_cm.assign(current_cm)

In [12]:
def get_one_hot(label, num_classes):
    """Get one hot tensor.

    Parameters
    ----------
    label: Numpy array.
        label.
    num_classes: Integer
        Number of classes.

    Returns
    -------
    One hot.
        Numpy array.
    """
    indexes = label.ravel()
    shape = tuple(list(label.shape) + [num_classes])
    onehot = np.zeros(shape=shape)
    onehot = onehot.ravel()

    for i in range(label.size):
        onehot[i * num_classes + indexes[i]] = 1

    onehot = onehot.reshape(shape)

    return onehot

In [4]:
# Initialize random generators.
seed = int(time.time())
seed = 1024
print(f'Seed:{seed}')
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

Seed:1024


In [5]:
%%writefile semantic_segmentation_deeplabv3plus_conf.json
{
	"mode" : "train",
	"resource_type": "pascal_voc_2012_ext",
	"resource_path" : "Z:\\maum\\workspace_resource\\deeplabv3plus_keras\\resource",
	"model_loading" : false,
	"multi_gpu" : false,
	"num_gpus" : 4,
	"eval_data_mode": 1,
	"eval_result_saving": false,
	"base_model": 0,
	"hps" : {
		"val_ratio": 0.1,
		"lr" : 0.0001,
		"beta_1" : 0.5,
		"beta_2" : 0.99,
		"decay" : 0.0,
		"epochs" : 2,
		"batch_size" : 1,
		"weight_decay": 0.00004,
		"bn_momentum": 0.9,
		"bn_scale": true,
		"reduce_lr_factor": 0.99
	},
	"nn_arch" : {
		"boundary_refinement": true,
		"output_stride": 16,
		"image_size": 512,
		"num_classes": 21,
		"mv2_depth_multiplier": 1,
		"depth_multiplier": 1,
		"conv_rate_multiplier" : 1,
		"reduction_size": 256,
		"dropout_rate": 0.5,
		"concat_channels": 256,
		"encoder_middle_conf": [
			{"kernel": 3, "rate": [1, 1], "op": "conv", "input": -1},
			{"kernel": 3, "rate": [18, 15], "op": "conv", "input": 0},
			{"kernel": 3, "rate": [6, 3], "op": "conv", "input": 1},
			{"kernel": 3, "rate": [1, 1], "op": "conv", "input": 0},
			{"kernel": 3, "rate": [6, 21], "op": "conv", "input": 0}
		],
		"encoder_middle_conf_xception": [
			{"kernel": 3, "rate": [1, 1], "op": "conv", "input": -1},
			{"kernel": 3, "rate": [6, 6], "op": "conv", "input": 0},
			{"kernel": 3, "rate": [12, 12], "op": "conv", "input": 0},
			{"kernel": 3, "rate": [18, 18], "op": "conv", "input": 0},
			{"kernel": 1, "rate": [1, 1], "op": "pyramid_pooling", "input": 0, "target_size_factor": [1, 1]}
		]
	}
}

Overwriting semantic_segmentation_deeplabv3plus_conf.json


In [7]:
with open("semantic_segmentation_deeplabv3plus_conf.json", 'r') as f:
    conf = json.load(f)

In [30]:
def cal_ss_class_imbalance_weights(resource_path, size=21):
    with open(os.path.join(resource_path
                           , 'VOCdevkit'
                           , 'VOC2012'
                           , 'ImageSets'
                           , 'Segmentation'
                           , 'train_aug_val.txt')) as f:
        file_names = f.readlines() #?
    
    # Remove \n.
    for i in range(len(file_names)):
        file_names[i] = file_names[i][:-1]
        
    label_dir_path = os.path.join(resource_path
                           , 'VOCdevkit'
                           , 'VOC2012'
                           , 'SegmentationClassAug')

    pf = np.zeros(size)
    total_num = 0.0

    for i in tqdm(range(len(file_names))):
        file_name = file_names[i]
        
        # Load label.
        label_path = os.path.join(label_dir_path, file_name + '.png') #?
        label = imread(label_path)
        
        label[label > (size - 1)] = 0        
        label_oh = get_one_hot(label, size)
        label2 = label_oh.reshape(np.prod(label.shape), size)
        label_pf = label2.sum(axis=0)
        pf = pf + label_pf
        total_num += np.prod(label.shape)
        
        if label_pf.sum() != np.prod(label.shape):
            print(f'{label_pf.sum()}, {np.prod(label.shape)}')

    pf = pf / total_num
    nf = 1.0 - pf
    pw = nf
    nw = pf

    print(f'pw: {pw}, nw: {nw}')
    return pw, nw

In [29]:
import pdb
pdb.runcall(cal_ss_class_imbalance_weights, conf['resource_path'])

> <ipython-input-26-9cb1f0cb93b8>(2)cal_ss_class_imbalance_weights()
      1 def cal_ss_class_imbalance_weights(resource_path, size=21):
----> 2     with open(os.path.join(resource_path
      3                            , 'VOCdevkit'
      4                            , 'VOC2012'
      5                            , 'ImageSets'

ipdb> tbreak 40
Breakpoint 8 at <ipython-input-26-9cb1f0cb93b8>:40
ipdb> c


100%|████████████████████████████████████████████████████▊| 11987/12031 [2:23:40<00:30,  1.45it/s]<ipython-input-26-9cb1f0cb93b8>:34: RuntimeWarning: overflow encountered in long_scalars
  total_num += np.prod(label.shape)
100%|█████████████████████████████████████████████████████| 12031/12031 [2:24:09<00:00,  1.39it/s]


Deleted breakpoint 8 at <ipython-input-26-9cb1f0cb93b8>:40
> <ipython-input-26-9cb1f0cb93b8>(40)cal_ss_class_imbalance_weights()
     38 
     39     pf = pf / total_num
---> 40     nf = 1.0 - pf
     41     pw = nf
     42     nw = pf

ipdb> pf
array([-0.70755247, -0.00899598, -0.00769173, -0.00883414, -0.00654323,
       -0.00548468, -0.01280812, -0.01923425, -0.03139148, -0.01255932,
       -0.00628141, -0.01065069, -0.02797298, -0.00926101, -0.01163078,
       -0.07447052, -0.00611032, -0.00630222, -0.01226675, -0.0135008 ,
       -0.00772092])
ipdb> total_num
-2139712417
ipdb> pf = pf * total_num
ipdb> of
*** NameError: name 'of' is not defined
ipdb> pf
array([1.51395881e+09, 1.92488160e+07, 1.64580820e+07, 1.89025120e+07,
       1.40006320e+07, 1.17356420e+07, 2.74056960e+07, 4.11557630e+07,
       6.71687460e+07, 2.68733240e+07, 1.34404170e+07, 2.27894140e+07,
       5.98541240e+07, 1.98158910e+07, 2.48865300e+07, 1.59345498e+08,
       1.30743380e+07, 1.34849450e+07, 2.62473170

In [31]:
pw, nw = cal_ss_class_imbalance_weights(conf['resource_path'])

100%|█████████████████████████████████████████████████████| 12031/12031 [1:52:41<00:00,  1.78it/s]

pw: [0.29754999 0.99106889 0.99236374 0.99122957 0.99350396 0.99455487
 0.98728424 0.98090446 0.96883489 0.98753125 0.99376389 0.98942612
 0.97222875 0.99080578 0.98845309 0.92606652 0.99393374 0.99374322
 0.98782171 0.98659656 0.99233476], nw: [0.70245001 0.00893111 0.00763626 0.00877043 0.00649604 0.00544513
 0.01271576 0.01909554 0.03116511 0.01246875 0.00623611 0.01057388
 0.02777125 0.00919422 0.01154691 0.07393348 0.00606626 0.00625678
 0.01217829 0.01340344 0.00766524]
